## Data collection toolkit for monitoring "peer effects"

This cell sets up the notebook to import numpy, datascience, seaborn, pandas, matplotlib etc.

In [196]:
# Run this cell to set up the notebook.

# These lines import the Numpy, Datascience, pandas modules.
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt

# Importing plotting libraries and styles
%matplotlib inline
plt.style.use('fivethirtyeight')

# For Pandas to ignore FutureWarning displays
import warnings
warnings.simplefilter('ignore', FutureWarning)

Defining the class `Person`

In [197]:
class Person:
    def __init__(self, index, startTime, neighbors, fromStart):
        self.index = index
        self.startTime = startTime
        self.breakStart = []
        self.breakEnd = []
        self.fromStart = int(fromStart)
        self.endTime = None
        self.tillEnd = 0
        self.neighbors = neighbors
    
    def addBreak(self, time):
        self.breakStart.append(time)
        return "Break started for person " + str(self.index) + " at " + str(time)
    
    def endBreak(self, time):
        if (len(self.breakStart)-1)!=len(self.breakEnd):
            return "Please start a break to stop it."
        else:
            self.breakEnd.append(time)
            return "Break ended for person " + str(self.index) + " at " + str(time)
        
    def endSession(self, time):
        self.endTime = time
        return "Person " + str(self.index) + " left at " + str(time)
        
    def numBreaks(self):
        return len(self.breakEnd)
    
    def getNeighbors(self):
        return self.neighbors
    
    def addNeighbor(self, number):
        self.neighbors.append(number)
        
    def setTillEnd(self, time):
        self.endTime = time
        self.tillEnd = 1

Defining the class `Table`

Todo:
* Add currentPersons list (or dict)
* Change self.persons to self.allPersons
* Add addNeighbor and removeNeighbor functionality
* Add this functionality to each of the Table functions and endSession functions etc.
* Change the export to the dataframe (especially the Neighbor part)


SOLVE UNIQUENESS PROBLEM with dictionary (`addBreak`)

In [198]:
class Table:
    # Todo think about this
    listTables = dict()
    
    def __init__(self, name, libraryName = ''):
        if (not isinstance(name, str)):
            print("ERROR: Table name should be a string")
            return
        if (not isinstance(libraryName, str)):
            print("ERROR: Table name should be a string")
            return
        self.name = name
        self.startTime = dt.datetime.now()
        self.library = libraryName
        
        # Dictionary of current active persons on Table (Number -> Person Object)
        self.currentPersons = dict()
        
        # List of Person Objects which have ever sat at Table
        self.allPersons = []
        
        Table.listTables[name] = self
        print("Table with name " + str(name) + " created at time: " + str(self.startTime))
        
    def addPerson(self, number, fromTheStart = False):
        if (not isinstance(number, int)):
            print("ERROR: Please give a number as person index")
            return
        if (not isinstance(fromTheStart, bool)):
            print("ERROR: Third argument should be True/False (person sitting from the start or not)")
            return
        if number in self.currentPersons:
            print("ERROR: Person already exists on Table: " + self.name)
            return
        neighbors = [key for key in self.currentPersons]
        
        # Adding current number as a neighbor to already existing neighbors
        for aPerson in self.currentPersons:
            self.currentPersons[aPerson].addNeighbor(number)
            
        # Creating Person object for this person
        newPerson = Person(number, dt.datetime.now(), neighbors, int(fromTheStart))
        self.currentPersons[number] = newPerson
        self.allPersons.append(newPerson)
        
        print("Person number " + str(number) + " added to Table " + str(self.name))
        return
    
    def addBreak(self, number):
        if (not isinstance(number, int)):
            print("ERROR: Please give a number as person index")
            return
        if number not in self.currentPersons:
            print("ERROR: Person " + str(number) + " not in list of active people on Table " + self.getName())
            return
        person = self.currentPersons[number]
        time = dt.datetime.now()
        print(person.addBreak(time))
        
    def getName(self):
        return self.name
        
    def endBreak(self, number):
        if (not isinstance(number, int)):
            print("ERROR: Please give a number as person index")
            return
        if number not in self.currentPersons:
            print("ERROR: Person " + str(number) + " not in list of people.")
            return
        time = dt.datetime.now()
        person = self.currentPersons[number]
        print(person.endBreak(time))
        
    def removePerson(self, number):
        if (not isinstance(number, int)):
            print("ERROR: Please give a number as person index")
            return
        if number not in self.currentPersons:
            print("This person doesn't exist and so, can't be removed")
            return
        else:
            person = self.currentPersons.pop(number)
            time = dt.datetime.now()
            print(person.endSession(time))
            return
    
    def getNumBreaks(self, number):
        if (not isinstance(number, int)):
            print("ERROR: Please give a number as person index")
            return
        if number not in self.currentPersons:
            print("ERROR: Person not active on Table")
            return
        person = self.currentPersons[number]
        print("Person " + str(number) + " has taken " + str(person.numBreaks()) + " breaks.")
        return
    
    def getCurrentPersons(self):
        return self.currentPersons

Creating the `convertToRow` function for appending to Dataframe

In [199]:
def convertToRow(person, table):
    row = [person.index, table.name, table.library, person.startTime, \
           person.endTime, person.numBreaks(), person.breakStart, person.breakEnd, person.fromStart, person.tillEnd, person.getNeighbors()]
    return row

The function `showTables` shows all the active Tables and active Seats on that table

In [200]:
def showTables():
    for table in Table.listTables.values():
        seatsTaken = table.getCurrentPersons().keys()
        print("Table Name: " + table.getName() + ", " + "Current seats taken: " + " ".join(str(e) for e in seatsTaken))

Run the function `quitSession` when you (the **Data Collector** want to leave)

In [201]:
def quitSession():
    time = dt.datetime.now()
    for table in Table.listTables.values():
        for person in table.currentPersons.values():
            person.setTillEnd(time)
    print("You left at", time)

## Extended Demo

In [202]:
# The code below creates a new table named A in Moffit Library
A = Table("A", "Moffit")

Table with name A created at time: 2019-03-08 21:58:47.432120


In [203]:
# Adds person at Seats 2,5 to Table A

A.addPerson(2)

A.addPerson(5)

# Trying to add Person 2 again
A.addPerson(2)

Person number 2 added to Table A
Person number 5 added to Table A
ERROR: Person already exists on Table: A


In [204]:
# Person 5 on Table A leaves for a break

A.addBreak(5)

Break started for person 5 at 2019-03-08 21:58:50.874176


In [205]:
# Similarly, creating Table B at Main Stacks

B = Table("B", "Stacks")

Table with name B created at time: 2019-03-08 21:58:52.680137


Adding person at Seat 1, Table B, but he was already there from the start. **Note:** *Adding True as the second argument implies that the person was there before you came in.

In [206]:
#Adding person at Seat 1, Table B, but he was already there from the start.

B.addPerson(1, True)

Person number 1 added to Table B


In [207]:
# Another person comes in and sits down at Seat 3, Table B.

B.addPerson(3)

Person number 3 added to Table B


In [208]:
# Person 1 and 3 on Table B leave together for a break

B.addBreak(1)

B.addBreak(3)

Break started for person 1 at 2019-03-08 21:58:55.383331
Break started for person 3 at 2019-03-08 21:58:55.383422


In [209]:
# 2 never left for a break
A.endBreak(2)

# But 5 did, so we can end his break.
A.endBreak(5)

Please start a break to stop it.
Break ended for person 5 at 2019-03-08 21:58:55.595837


In [210]:
# Person 5 on Table A departs for the day

A.removePerson(5)

Person 5 left at 2019-03-08 21:58:56.154816


In [211]:
# Trying to remove Person 5 again.
A.removePerson(5)

This person doesn't exist and so, can't be removed


In [212]:
# Showing all the Seats that are currently active

showTables()

Table Name: A, Current seats taken: 2
Table Name: B, Current seats taken: 1 3


In [213]:
# Removing Person 1 from Table B

B.removePerson(1)

Person 1 left at 2019-03-08 21:58:59.128115


In [214]:
# Again, showing active seats

showTables()

Table Name: A, Current seats taken: 2
Table Name: B, Current seats taken: 3


In [215]:
# Suppose I want to leave the library now, I simply run

quitSession()

You left at 2019-03-08 21:59:00.895141


## Start writing your code below to add Tables

### Important (Read me)
1. Run the cell below to update the libData.csv file!

**Format:** Person, Table, Library, Arrival Time, Departure Time, Num Breaks, Break Start Times (list), Break End Times (list), fromStart, tillEnd, Neighbours

In [216]:
# RUN THIS CELL AT THE END, then close this tab!

# This automatically adds data that we've input into the libData.csv dataframe
df = pd.read_csv('libData.csv')
for table in Table.listTables.values():
    for person in table.allPersons:
        df = df.append(pd.DataFrame([convertToRow(person, table)],columns=df.columns))
df.to_csv('libData.csv', index=False)